# Visualize AFOG Attack on DETR
This notebook provides functions for visualizing DETR and its self-attention weights before and after attacking a single COCO image. Please following setup instructions in README.md prior to running this notebook.

In [ ]:
import os
import sys
sys.path.append(os.getcwd()[:os.getcwd().find("demos")])
import torch, json
import numpy as np

from utils.detr_utils import build_model
from utils.dino_utils.slconfig import SLConfig
from utils.dataset_utils.coco import build_dataset
from utils.dino_utils.visualizer import COCOVisualizer
from utils.dino_utils import box_ops
import matplotlib.pyplot as plt
import numpy as np
import random
%matplotlib inline

%load_ext autoreload
%autoreload 2

Helper functions for preparing and displaying one image are in the cell below.

In [ ]:
# adapted from https://colab.research.google.com/github/facebookresearch/detr/blob/colab/notebooks/detr_attention.ipynb#scrollTo=eg4RK8JiYTEl
import torchvision.transforms as T
def plot_results(pil_img, prob, boxes):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        cl = p.argmax()
        text = f'{CLASSES[cl]}: {p[cl]:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15,
                bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    plt.show()
    
# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32).cuda()
    return b


def visualize_decoder_weights(model, img, mode="single", caption="Image"):
    transform = T.Compose([
        T.Resize(800),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
    ]
    COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

    # propagate through the model
    outputs = model(img.unsqueeze(0).cuda())

    # keep only predictions with 0.7+ confidence
    probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
    keep = probas.max(-1).values > 0.3

    # convert boxes from [0; 1] to image scales
    print("Size is", img.shape)
    bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], (img.shape[2], img.shape[1]))
    
    # use lists to store the outputs via up-values
    conv_features, enc_attn_weights, dec_attn_weights = [], [], []

    hooks = [
        model.backbone[-2].register_forward_hook(
            lambda self, inp, output: conv_features.append(output)
        ),
        model.transformer.encoder.layers[-1].self_attn.register_forward_hook(
            lambda self, inp, output: enc_attn_weights.append(output[1])
        ),
        model.transformer.decoder.layers[-1].multihead_attn.register_forward_hook(
            lambda self, inp, output: dec_attn_weights.append(output[1])
        ),
    ]
    
    # propagate through the model
    outputs = model(img.unsqueeze(0).cuda())

    for hook in hooks:
        hook.remove()

    # don't need the list anymore
    conv_features = conv_features[0]
    enc_attn_weights = enc_attn_weights[0]
    dec_attn_weights = dec_attn_weights[0]
    
    # get the feature map shape
    h, w = conv_features['0'].tensors.shape[-2:]

    print("There are", len(bboxes_scaled), "objects in the figure")
    img = img.permute(1, 2, 0)
    img = (img - torch.min(img)) / (torch.max(img) - torch.min(img))
    plt.clf()
    
    if mode == "multi":
        fig, ax = plt.subplots(ncols=2, nrows=len(bboxes_scaled), figsize=(2*5, len(bboxes_scaled) * 5))
        colors = COLORS * 100
        i = 0

        for idx, (xmin, ymin, xmax, ymax) in zip(keep.nonzero(), bboxes_scaled.detach().cpu().numpy()):
            if len(bboxes_scaled) > 1:
        #         ax = ax_i[0]
                ax[i,0].imshow(dec_attn_weights[0, idx].view(h, w).detach().cpu().numpy())
                ax[i,0].axis('off')
                ax[i,0].set_title(f'query id: {idx.item()}')
                #ax = ax_i[1]
                ax[i,1].imshow(img)
                ax[i,1].add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                           fill=False, color='blue', linewidth=3))
                ax[i,1].axis('off')
                ax[i,1].set_title(CLASSES[probas[idx].argmax()])
                i += 1
            else:
                ax[0].imshow(dec_attn_weights[0, idx].view(h, w).detach().cpu().numpy())
                ax[0].axis('off')
                ax[0].set_title(f'query id: {idx.item()}')
                #ax = ax_i[1]
                ax[1].imshow(img)
                ax[1].add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                           fill=False, color='blue', linewidth=3))
                ax[1].axis('off')
                ax[1].set_title(CLASSES[probas[idx].argmax()])
                i += 1
        fig.tight_layout()
        plt.show()
    
    else:
        # Use the built-in COCO visualizer
        output = postprocessors['bbox'](outputs, torch.Tensor([[1.0, 1.0]]).cuda())[0]

        threshold = 0.5 # set a thershold

        scores = output['scores']
        labels = output['labels']
        boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
        select_mask = scores > threshold
        box_label = [id2name[int(item)] for item in labels[select_mask]]
        pred_dict = {
            'boxes': boxes[select_mask],
            'size': targets['size'],
            'box_label': box_label
        }
        vslzr = COCOVisualizer()
        vslzr.visualize(image, pred_dict, savedir=None, caption=caption)
 
        # Now plot the decoder's attention weights
        total_weights = np.zeros(dec_attn_weights[0, 0].view(h, w).detach().cpu().numpy().shape)
        for idx, (xmin, ymin, xmax, ymax) in zip(keep.nonzero(), bboxes_scaled.detach().cpu().numpy()):
            total_weights += dec_attn_weights[0, idx].view(h, w).detach().cpu().numpy()
            #ax[1].add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, color='blue', linewidth=3))

        plt.imshow(total_weights)
        plt.axis('off')
        plt.title("Decoder Attention Weights (" + caption + ")")
        plt.show()

def visualize_encoder_weights(model, img, idxs):
    transform = T.Compose([
        T.Resize(800),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
    ]
    COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

    # propagate through the model
    outputs = model(img.unsqueeze(0).cuda())

    # keep only predictions with 0.7+ confidence
    probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
    keep = probas.max(-1).values > 0.3

    # convert boxes from [0; 1] to image scales
    print("Size is", img.shape)
    bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], (img.shape[2], img.shape[1]))
    
    # use lists to store the outputs via up-values
    conv_features, enc_attn_weights, dec_attn_weights = [], [], []

    hooks = [
        model.backbone[-2].register_forward_hook(
            lambda self, inp, output: conv_features.append(output)
        ),
        model.transformer.encoder.layers[-1].self_attn.register_forward_hook(
            lambda self, inp, output: enc_attn_weights.append(output[1])
        ),
        model.transformer.decoder.layers[-1].multihead_attn.register_forward_hook(
            lambda self, inp, output: dec_attn_weights.append(output[1])
        ),
    ]
    
    # propagate through the model
    outputs = model(img.unsqueeze(0).cuda())

    for hook in hooks:
        hook.remove()

    # don't need the list anymore
    conv_features = conv_features[0]
    enc_attn_weights = enc_attn_weights[0]
    dec_attn_weights = dec_attn_weights[0]
    
    f_map = conv_features['0']
    print("Encoder attention:      ", enc_attn_weights[0].shape)
    print("Feature map:            ", f_map.tensors.shape)
    
    # get the HxW shape of the feature maps of the CNN
    shape = f_map.tensors.shape[-2:]
    # and reshape the self-attention to a more interpretable shape
    sattn = enc_attn_weights[0].reshape(shape + shape)
    print("Reshaped self-attention:", sattn.shape)
    
    # downsampling factor for the CNN, is 32 for DETR and 16 for DETR DC5
    fact = 32
    
    img = img.permute(1, 2, 0)
    img = (img - torch.min(img)) / (torch.max(img) - torch.min(img))

    # here we create the canvas
    plt.clf()
    fig = plt.figure(constrained_layout=True, figsize=(25 * 0.7, 8.5 * 0.7))
    # and we add one plot per reference point
    gs = fig.add_gridspec(2, 4)
    axs = [
        fig.add_subplot(gs[0, 0]),
        fig.add_subplot(gs[1, 0]),
        fig.add_subplot(gs[0, -1]),
        fig.add_subplot(gs[1, -1]),
    ]

    # for each one of the reference points, let's plot the self-attention
    # for that point
    for idx_o, ax in zip(idxs, axs):
        idx = (idx_o[0] // fact, idx_o[1] // fact)
        ax.imshow(sattn[..., idx[0], idx[1]].detach().cpu().numpy(), cmap='cividis', interpolation='nearest')
        ax.axis('off')
        ax.set_title(f'self-attention{idx_o}')

    # and now let's add the central image, with the reference points as red circles
    fcenter_ax = fig.add_subplot(gs[:, 1:-1])
    fcenter_ax.imshow(img.detach().cpu().numpy())
    for (y, x) in idxs:
        scale = img.shape[0] / img.shape[-2]
        x = ((x // fact) + 0.5) * fact
        y = ((y // fact) + 0.5) * fact
        fcenter_ax.add_patch(plt.Circle((x * scale, y * scale), fact // 2, color='r'))
        fcenter_ax.axis('off')
    plt.show()
    
def visualize_attack(model, image, targets, attack, mode, n_iter=10):
    x_adv, etas, eta_grads, map_grads, maps = attack(model, image.unsqueeze(0).float(), vis=True, n_iter = n_iter, mode="vanishing")
    
    # normalize grads to make them easier to see
    perts = np.multiply(etas, maps)
    perts_norm = [(grad - np.min(grad)) / (np.max(grad) - np.min(grad)) for grad in perts]
    perts_norm = [np.clip(1 - np.abs(pert - np.median(pert)), 0, 1) for pert in perts_norm]
    
    eta_grads_norm = [(grad - np.min(grad)) / (np.max(grad)-np.min(grad)) for grad in eta_grads]
    eta_grads_norm = [np.clip(1 - 5*np.abs(grad - np.median(grad)), 0, 1) for grad in eta_grads_norm]
    
    map_grads_norm = [(grad - np.min(grad)) / (np.max(grad)-np.min(grad)) for grad in map_grads]
    map_grads_norm = [np.clip(1 - 5*np.abs(grad - np.median(grad)), 0, 1) for grad in map_grads_norm]
    
    etas_norm = [(eta - np.min(eta)) / (np.max(eta)-np.min(eta)) for eta in etas]
    etas_norm = [np.clip(1 - np.abs(eta - np.median(eta)), 0, 1) for eta in etas_norm]
    
    maps_norm = [(m - np.min(m)) / (np.max(m)-np.min(m)) for m in maps]
    maps_norm = [np.clip(1-np.abs(m - np.median(m)), 0.7, 1) for m in maps_norm]
    
    fig, ax = plt.subplots(nrows=n_iter, ncols=5, figsize=(15, n_iter*3+8))

    for i in range(n_iter):
#         if i == n_iter - 1:
#             ax[i][0].matshow(etas[i].squeeze()[:, :, 0])
#             ax[i][1].matshow(np.zeros(etas_norm[i].squeeze()[:, :, 0].shape))
#             ax[i][2].matshow(np.zeros(etas_norm[i].squeeze()[:, :, 0].shape))
#             ax[i][3].hist(grads[i].flatten(), bins=100)
#         else:
        #ax[i].matshow(etas[i].squeeze()[:, :, 0])
    
        colormap = "RdYlBu"
        ax[i][0].matshow(perts_norm[i].squeeze()[0, :, :], cmap=colormap)
        ax[i][1].matshow(eta_grads_norm[i].squeeze()[0, :, :], cmap=colormap)
        ax[i][2].matshow(maps_norm[i].squeeze()[0, :, :], cmap=colormap)
        ax[i][3].matshow(map_grads_norm[i].squeeze()[0, :, :], cmap=colormap)
        ax[i][4].matshow(etas_norm[i].squeeze()[0, :, :], cmap=colormap)
#             ax[i].set_title(str(i))
#             ax[i,:].set_axis_off()
        ax[i][0].set_title("After Iter " + str(i+1) + ": A*P.")
        ax[i][1].set_title("P Grad")
        ax[i][2].set_title("A Map")
        ax[i][3].set_title("A Grad")
        ax[i][4].set_title("P")
    plt.show()
    
# Visualize model predictions
def viz_preds(model, image, targets, caption):
    output = model(image[None].cuda())
    output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]).cuda())[0]

    thershold = 0.5 # set a thershold
    scores = output['scores']
    labels = output['labels']
    boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
    select_mask = scores > thershold
    box_label = [id2name[int(item)] for item in labels[select_mask]]
    pred_dict = {
        'boxes': boxes[select_mask],
        'size': targets['size'],
        'box_label': box_label
    }
    vslzr = COCOVisualizer()
    vslzr.visualize(image, pred_dict, savedir=None, caption=caption)
    
    

Select which version of DETR to use, either ResNet-50 or ResNet-101.

In [ ]:
# --- DETR-R50 ---
model_config_path = "../utils/detr_utils/config/r50_config.py"
model_checkpoint_path = "../model_files/detr-r50.pth"
args = SLConfig.fromfile(model_config_path) 
args.resume="resnet50"
args.backbone="resnet50"

# --- DETR-R101 ---
# model_config_path = "../utils/detr_utils/config/r50_config.py"
# model_checkpoint_path = "../model_files/detr-r101.pth"
# args = SLConfig.fromfile(model_config_path) 
# args.resume="resnet101"
# args.backbone="resnet101"

Prepare model and dataset.

In [ ]:
args.device = 'cuda'
args.masks = False
model, criterion, postprocessors = build_model(args)
checkpoint = torch.load(model_checkpoint_path, map_location='cpu', weights_only=True)
_ = model.eval()
model.postprocessors = postprocessors
model.load_state_dict(checkpoint['model'])
model.to(torch.device('cuda'))
model.criterion = criterion
# load coco names
with open('../utils/dino_utils/coco_id2name.json') as f:
    id2name = json.load(f)
    id2name = {int(k):v for k,v in id2name.items()}
    
args.dataset_file = 'coco'
args.coco_path = "../datasets/coco/" # the path of coco
args.fix_size = False

dataset_val = build_dataset(image_set='val', args=args)  

Display the benign image, benign prediction, DETR's encoder's last layer self-attention weights, and DETR decoder's last layer's weights at the points in idxs. Maximum of four points supported.

In [ ]:
IM_NUM=500 #which image to attack, up to 4999
idxs = [(380, 1600), (400, 300),] #points in the image to investigate self-attention

image, targets = dataset_val[IM_NUM]
print("--- Image: ", targets["image_id"][0].item(), "---")

visualize_decoder_weights(model, image, mode="single", caption="")
visualize_encoder_weights(model, image, idxs)
viz_preds(model, image, targets, caption="")

Attack the image with AFOG and display the adversarial prediction as well as DETR's encoder's last layer self-attention weights at the same points. Attack mode can be changed by setting mode to either "baseline", "vanishing", or "fabrication."

In [ ]:
from afog.attacks import afog

# Attack the image
image_afog = afog(model, image.unsqueeze(0).float(), n_iter=10, mode="baseline")

viz_preds(model, image_afog.squeeze().float(), targets, caption="")
visualize_decoder_weights(model, image_afog.squeeze().float(), mode="singer", caption="")
visualize_encoder_weights(model, image_afog.squeeze().float(), idxs)

Visualize the internals of AFOG as it iterates, including perturbation map, attention map, and their gradients.

In [ ]:
visualize_attack(model, image, targets, afog, "baseline")

### 

### 

### 